In [18]:
import torch
import numpy as np
import os
import sys
import random
from PIL import Image
from torch.utils.data import Dataset

In [6]:
import torch.nn as nn
import torch.nn.functional as F

FC1_SIZE = 200
FC2_SIZE = 100

class MLP(nn.Module):
    def __init__(self, classes, input_size):
        super(MLP, self).__init__()

        self.hidden1_output = None
        self.hidden2_output = None

        FC1_SIZE = 2*input_size
        print("FC1_SIZE is :{}\nFC2_SIZE is {}".format(FC1_SIZE, FC2_SIZE))

        # =======================无trick=======================
        self.fc1 = nn.Linear(input_size, FC1_SIZE)
        self.fc2 = nn.Linear(FC1_SIZE, FC2_SIZE)
        self.fc3 = nn.Linear(FC2_SIZE, classes)

        # # =======================tricks: dropout, BN=======================
        # self.fc1 = nn.Linear(input_size, FC1_SIZE)
        # self.bn1 = nn.BatchNorm1d(num_features=FC1_SIZE)
        #
        # self.dropout = nn.Dropout(p=0.3)
        #
        # self.fc2 = nn.Linear(FC1_SIZE, FC2_SIZE)
        # self.bn2 = nn.BatchNorm1d(num_features=FC2_SIZE)
        # self.fc3 = nn.Linear(FC2_SIZE, classes)



        # self.layers = nn.Sequential(
        #     nn.Linear(input_size, 200),
        #     nn.BatchNorm1d(num_features=200),
        #     nn.ReLU(),
        #     nn.Linear(200, 100),
        #     nn.BatchNorm1d(num_features=100),
        #     nn.ReLU(),
        #     nn.Linear(100, classes)
        # )


    def forward(self, x):
        '''Forward pass'''
        # =======================无trick=======================
        x = self.fc1(x)
        self.hidden1_output = x
        x = F.relu(x)

        x = self.fc2(x)
        self.hidden2_output = x
        x = F.relu(x)

        x = self.fc3(x)

        # # =======================tricks: dropout, BN=======================
        # x = self.fc1(x)
        # # 记录hidden1的输出
        # self.hidden1_output = x
        # x = self.bn1(x)
        # x = F.relu(x)
        #
        # x = self.fc2(x)
        # # 记录hidden1的输出
        # self.hidden2_output = x
        # x = self.bn2(x)
        # x = F.relu(x)
        #
        # x = self.dropout(x)
        #
        # x = self.fc3(x)

        return x, self.hidden1_output, self.hidden2_output

    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight.data, 0, 0.1)
                m.bias.data.zero_()

# 加载模型

In [8]:
net_new = MLP(classes=10, input_size=76)

FC1_SIZE is :152
FC2_SIZE is 100


In [11]:
path_state_dict = "../saved_model/mfeat-fou.pkl"
state_dict_load = torch.load(path_state_dict)

In [13]:
print("加载前: ", net_new.fc1.weight[0, ...])
net_new.load_state_dict(state_dict_load)
print("加载后: ", net_new.fc1.weight[0, ...])


加载前:  tensor([-0.1069, -0.0318,  0.1046, -0.0002,  0.0290,  0.0969, -0.0630, -0.0435,
         0.0404,  0.1044,  0.1145,  0.0877, -0.0581,  0.0917,  0.0379,  0.0802,
         0.0784,  0.0308,  0.0006, -0.0526, -0.0963,  0.0969,  0.1028,  0.0729,
        -0.1120, -0.0436,  0.0858,  0.0588,  0.0490, -0.0831, -0.1055,  0.0364,
         0.0901, -0.0278,  0.0084, -0.0118,  0.0609,  0.0437, -0.0938,  0.0346,
         0.0993, -0.0032,  0.0021,  0.0947,  0.0760,  0.1101, -0.0608, -0.0831,
        -0.1041, -0.0659,  0.0309, -0.0973,  0.0328,  0.1081,  0.1102, -0.0674,
         0.0190,  0.0308, -0.0223,  0.0507, -0.0846,  0.0884, -0.1124, -0.0528,
         0.0376,  0.0668,  0.0209, -0.0639, -0.0837,  0.0069,  0.1054, -0.0990,
         0.0433,  0.0845, -0.1128,  0.0860], grad_fn=<SelectBackward>)
加载后:  tensor([ 0.1766,  0.1869,  0.0384, -0.0698, -0.1771,  0.1209, -0.0867,  0.1388,
         0.0137,  0.0360, -0.0255, -0.0432,  0.0514, -0.0565, -0.1049, -0.1762,
         0.0926,  0.1054,  0.0993, -0

# 批处理加载模型,dataloader

In [28]:
import numpy as np
import torch
import os
import sys
import random
from PIL import Image
from torch.utils.data import Dataset

DATASIZE = 2000
# !!!如果改变此参数,split_datasets中的也需要改
TRAIN_PRECENT = 0.8
CLASS_NUM = 10


class handWritten_Dataset(Dataset):
    def __init__(self, data_path, transform=None, train=True):
        """
        mfeat-fou的Dataset
        :param data_path: str, 数据集所在路径
        :param transform: torch.transform，数据预处理
        """

        self.label_name = {"0": 0, "1": 1}
        self.class_size = self.get_class_size(train)
        self.data_info = self.get_img_info(data_path)
        self.transform = transform


    def __getitem__(self, index):
        data, label = self.data_info[index]

        if self.transform is not None:
            data = self.transform(data)

        return data, label

    def __len__(self):
        return len(self.data_info)

    # @staticmethod
    def get_class_size(self, train):
        class_size = int(DATASIZE*TRAIN_PRECENT/CLASS_NUM)
        if not train:
            class_size = 200 - class_size
        return class_size

    # @staticmethod
    def get_img_info(self, file_path):
        data_info = list()
        train_data = np.load(file_path)
        for i, data in enumerate(train_data):
            label = int(i / self.class_size)
            data_info.append((data, label))

        # print(data_info[0])
        return data_info

    def get_features_size(self):
        return len(self.data_info[0][0])




In [26]:
# 批处理
featureSet_list = ['mfeat-fou', 'mfeat-fac', 'mfeat-kar', 'mfeat-pix', 'mfeat-zer']

## 加载到list中

In [30]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_set = list()
dataset_set = list()
dataloader_set = list()
for i, file_name in enumerate(featureSet_list):
    train_file_path = '../datasets/Handwritten/mfeat/splite/' + file_name + '/train_data.npy'
    model_path = "../saved_model/" + file_name + ".pkl"

    dataset_set.append(handWritten_Dataset(data_path=train_file_path))
#     train_loader = torch.utils.data.DataLoader(dataset=dataset_set[i], batch_size=BATCH_SIZE, shuffle=True)

    model_set.append(MLP(classes=10, input_size=dataset_set[i].get_features_size()).to(device))
    state_dict_load = torch.load(model_path)
    model_set[i].load_state_dict(state_dict_load)
    print("加载后: ", model_set[i].fc1.weight[0, ...])

FC1_SIZE is :152
FC2_SIZE is 100
加载后:  tensor([ 0.1766,  0.1869,  0.0384, -0.0698, -0.1771,  0.1209, -0.0867,  0.1388,
         0.0137,  0.0360, -0.0255, -0.0432,  0.0514, -0.0565, -0.1049, -0.1762,
         0.0926,  0.1054,  0.0993, -0.1239,  0.0232,  0.0411, -0.0545,  0.1181,
        -0.1075,  0.1310,  0.1688, -0.0031,  0.1419, -0.0905,  0.0811, -0.0861,
         0.1768, -0.0319, -0.1411,  0.0402,  0.0159, -0.0238, -0.0191, -0.0171,
        -0.1630,  0.0764, -0.0944,  0.1212, -0.1823,  0.0376, -0.0301, -0.1606,
         0.1330,  0.0614,  0.0405,  0.2159,  0.0509, -0.0285,  0.0965,  0.0884,
         0.1265, -0.0923,  0.0593, -0.0384, -0.0522, -0.0790,  0.1762, -0.0917,
         0.0624,  0.0614, -0.0761, -0.0051, -0.0876,  0.1221, -0.1156, -0.0207,
        -0.0894, -0.0218, -0.0139,  0.2351], device='cuda:0',
       grad_fn=<SelectBackward>)
FC1_SIZE is :432
FC2_SIZE is 100
加载后:  tensor([ 8.5771e-02,  8.2893e-02,  2.9989e-04,  2.5167e-01, -3.9782e-02,
        -1.3033e-01,  3.7071e-02, 

In [31]:
print(len(dataset_set[0]))

1600


In [37]:
torch.rand((1600,64))

tensor([[0.6687, 0.7048, 0.4109,  ..., 0.7064, 0.2379, 0.9506],
        [0.7398, 0.6648, 0.9866,  ..., 0.0376, 0.3576, 0.1309],
        [0.9410, 0.3109, 0.5236,  ..., 0.8389, 0.4405, 0.5823],
        ...,
        [0.7375, 0.5014, 0.7620,  ..., 0.5307, 0.4031, 0.8104],
        [0.2523, 0.6842, 0.1313,  ..., 0.1358, 0.8581, 0.6817],
        [0.1029, 0.5743, 0.6792,  ..., 0.4843, 0.8884, 0.3806]])